In [29]:
import numpy as np
import time
np.set_printoptions(precision=3, suppress=True)

NUMBER_OF_SOURCE = 3

NUMBER_OF_LINK = 8

S_and_L = np.asarray(
    [[1, 1, 0, 1, 0, 0, 1, 0],
     [1, 0, 1, 0, 1, 0, 0, 0, ],
     [0, 0, 0, 0, 0, 1, 1, 1]]
)

COLLISION_GRAPH = np.asarray(
    [[0, 1, 1, 1, 1, 0, 0, 0],
     [1, 0, 1, 1, 0, 0, 1, 0],
     [1, 1, 0, 0, 1, 0, 0, 0],
     [1, 1, 0, 0, 0, 1, 1, 1],
     [1, 0, 1, 0, 0, 0, 0, 1],
     [0, 0, 0, 1, 0, 0, 1, 1],
     [0, 1, 0, 1, 0, 1, 0, 1],
     [0, 0, 0, 1, 1, 1, 1, 0]], dtype=np.int32
)

#表示冲突图

C_L = 10 * np.ones((NUMBER_OF_LINK, 1)) #最大速率

C_L = np.asarray([[10,10,10,10,10,10,10,10]]).T
DELAY_S = np.asarray([[10, 10, 10]]).T # 时延限制

average_cl = np.zeros((NUMBER_OF_LINK,1))
speed_s = np.ones((NUMBER_OF_SOURCE, 1))
sigma_l = np.ones((NUMBER_OF_LINK, 1))
p_l = np.ones((NUMBER_OF_LINK, 1))
mu_s = np.ones((NUMBER_OF_SOURCE, 1))

In [30]:

links_work_at_the_same_time = np.asarray(
    [
        [1,0,0,0,0,1,0,0],
        [1,0,0,0,0,0,1,0],
        [1,0,0,0,0,0,0,1],
        [0,1,0,0,0,1,0,0],
        [0,1,0,0,0,0,0,1],
        [0,0,1,0,0,1,0,0],
        [0,0,1,1,0,0,0,0],
        [0,0,1,0,0,0,0,1],
        [0,0,0,1,1,0,0,0],
        [0,0,0,0,1,1,0,0],
        [0,0,0,0,1,0,1,0],
    ],dtype=np.int32

)


In [31]:
S_and_L # 表示源经过的链路
COLLISION_GRAPH  #表示冲突图
C_L #最大速率
DELAY_S# 时延限制
links_work_at_the_same_time # 同一时刻能够工作的节点

array([[1, 0, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 0]])

In [32]:
k = 0
delta = 1
beta = 0.01
gama = 0.01

average_pl = np.zeros((NUMBER_OF_LINK, 1))
average_mu_s = np.zeros((NUMBER_OF_SOURCE, 1))
a = time.time()
while k<10000:
    print("每条链路的平均速率：", average_cl.T[0])  # 每条链路的平均速度
    print("p_{l}的平均值：   ",average_pl.T[0])
    print("mu_{s}的平均值：  ",average_mu_s.T[0])
    print("当前源速率:       ",speed_s.T[0])# 源速率
    # print(sigma_l.T )# 每条链路的裕量
    # print(p_l.T )# 每条链路的拉格朗日乘子
    # print(mu_s .T)# 每个源的乘子

    print('\n\n\n')

    speed_s = 1/S_and_L.dot(p_l)
    sigma_l = np.sqrt( S_and_L.T.dot(mu_s)/p_l)

    tmp = links_work_at_the_same_time.dot(p_l).sum(axis=1)

    work_mode = links_work_at_the_same_time[np.argmax(tmp)].reshape((NUMBER_OF_LINK,1))

    k+= delta
    average_cl = (average_cl*(k-1) + delta*C_L* work_mode)/k



    p_l = p_l + beta * (S_and_L.T.dot(speed_s) - average_cl + sigma_l)

    mu_s = mu_s + gama *(S_and_L.dot(1/sigma_l)-DELAY_S)


    p_l = np.maximum(p_l, 0.01)
    mu_s= np.maximum(mu_s, 0.01)
    average_pl = (average_pl*(k-1) + p_l)/k
    average_mu_s = (average_mu_s*(k-1) + mu_s)/k

print(str(time.time()-a))


每条链路的平均速率： [0. 0. 0. 0. 0. 0. 0. 0.]
p_{l}的平均值：    [0. 0. 0. 0. 0. 0. 0. 0.]
mu_{s}的平均值：   [0. 0. 0.]
当前源速率:        [1. 1. 1.]




每条链路的平均速率： [10.  0.  0.  0.  0. 10.  0.  0.]
p_{l}的平均值：    [0.92  1.012 1.013 1.012 1.013 0.913 1.02  1.013]
mu_{s}的平均值：   [0.934 0.927 0.927]
当前源速率:        [0.25  0.333 0.333]




每条链路的平均速率： [5. 0. 0. 0. 5. 5. 5. 0.]
p_{l}的平均值：    [0.905 1.019 1.02  1.019 0.995 0.895 1.005 1.02 ]
mu_{s}的平均值：   [0.902 0.891 0.891]
当前源速率:        [0.252 0.339 0.339]




每条链路的平均速率： [3.333 0.    3.333 0.    3.333 3.333 3.333 3.333]
p_{l}的平均值：    [0.896 1.025 1.015 1.025 0.982 0.882 0.995 1.015]
mu_{s}的平均值：   [0.87  0.855 0.855]
当前源速率:        [0.255 0.346 0.346]




每条链路的平均速率： [2.5 0.  5.  2.5 2.5 2.5 2.5 2.5]
p_{l}的平均值：    [0.89  1.03  1.003 1.024 0.972 0.873 0.988 1.01 ]
mu_{s}的平均值：   [0.838 0.82  0.82 ]
当前源速率:        [0.255 0.352 0.352]




每条链路的平均速率： [2. 2. 4. 2. 2. 2. 2. 4.]
p_{l}的平均值：    [0.886 1.032 0.991 1.022 0.965 0.866 0.984 1.001]
mu_{s}的平均值：   [0.807 0.785 0.785]
当前